# Experimenty s Iterovanou dilatáciou

**Autor: Bc. Ivan Vykopal**

Tento notebook obsahuje funkcie pre experimenty s iterovanou dilatáciou pre identifikovanie záplových ložísk v histologických snímok na základe anotácií a segmentácii buniek.

In [ ]:
import pandas as pd
import copy
import geojson
from geojson import Polygon, FeatureCollection, Feature, dump
import numpy as np
import os 
from shapely.geometry import shape, Polygon, MultiPolygon
import matplotlib.pyplot as plt
import random 
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering, DBSCAN
from scipy.cluster import hierarchy
import cv2 as cv
import json

import javabridge
import bioformats as bf

from os.path import exists
import matplotlib.pyplot as plt
import geopandas as gpd

import itertools
from shapely.geometry import mapping
from shapely.ops import nearest_points

from scipy.spatial import cKDTree

In [ ]:
def display_sample(display_list):
    plt.figure(figsize=(18, 18))

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.imshow(display_list[i])
        plt.axis('off')
    plt.show()

In [ ]:
def evaluate(original_mask, predicted_mask):
    overlap = original_mask * predicted_mask
    union = original_mask + predicted_mask
    union = np.clip(union, 0, 1)
    
    if overlap.sum() == 0 and union.sum() == 0:
        return 1, 1
    
    IOU = overlap.sum() / float(union.sum())
    
    dice = 2 * (overlap.sum()) / (union.sum() + overlap.sum())
    
    return IOU, dice

In [ ]:
def set_logs():
    myloglevel = "ERROR"
    rootLoggerName = javabridge.get_static_field("org/slf4j/Logger", "ROOT_LOGGER_NAME", "Ljava/lang/String;")
    rootLogger = javabridge.static_call("org/slf4j/LoggerFactory", "getLogger",
                                        "(Ljava/lang/String;)Lorg/slf4j/Logger;", rootLoggerName)
    logLevel = javabridge.get_static_field("ch/qos/logback/classic/Level", myloglevel,
                                           "Lch/qos/logback/classic/Level;")
    javabridge.call(rootLogger, "setLevel", "(Lch/qos/logback/classic/Level;)V", logLevel)

In [ ]:
def find_tissues(gj):
    features = gj['features']

    for feature in features:
        class_type = feature['properties']['classification']['name']
        if class_type == 'Region*':
            return feature

In [ ]:
def get_fragment_coords(gj):
    if gj:
        features = gj['features']
        tissues = find_tissues(gj)['geometry']['coordinates']
        main_features_len = len(tissues)
        main_coordinates = list()

        for i in range(0, main_features_len):
            coordinates = tissues[i]
            geo: dict = {'type': 'Polygon', 'coordinates': coordinates}
            polygon: Polygon = shape(geo)
            mabr = polygon.bounds
            mabr = [int(x) for x in mabr]
            main_coordinates.append(mabr)

        return main_coordinates, gj
    else:
        return None, None

In [ ]:
def get_geojson_from_file(path):
    try:
        return geojson.load(open(path))
    except OSError:
        return None

In [ ]:
def pretize_text(annotation_type):
    if annotation_type == 'blood_vessels':
        return 'Blood vessels'
    elif annotation_type == 'fatty_tissues':
        return 'Fatty tissue'
    elif annotation_type == 'inflammations':
        return 'Inflammation'
    elif annotation_type == 'endocariums':
        return 'Endocarium'
    elif annotation_type == 'fibrotic_tissues':
        return 'Fibrotic tissue'
    elif annotation_type == 'quilities':
        return 'Quilty'
    elif annotation_type == 'immune_cells':
        return 'Immune cells'
    else:
        annotation_type = annotation_type.replace('_', ' ')
        return annotation_type.replace(annotation_type[0], annotation_type[0].upper(), 1)

In [ ]:
def get_coors(contour):
    coors = []
    for idx in range(len(contour)):
        coors.append(contour[idx, 0].tolist())

    return coors

In [ ]:
def fix_polygon(contour):
    return np.concatenate((contour, [contour[0]]))

In [ ]:
def create_properties_template(annotation):
    return {
        "object_type": "annotation",
        "classification": {
            "name": pretize_text(annotation)
        },
    }

In [ ]:
def create_polygon(coors):
    return {
        "type": "Polygon",
        "coordinates": coors
    }

In [ ]:
def get_features(contours, annotation):
    features = []
    for contour in contours:
        contour = fix_polygon(contour)
        coors = get_coors(contour)
        if len(coors) < 3:
            continue

        features.append(Feature(
            geometry=create_polygon([coors]),
            properties=create_properties_template(annotation)
        ))

    return features

In [ ]:
def create_geojson(mask, annotation_classes):
    mask = np.uint8(mask)

    features = []
    if len(mask.shape) == 3:
        _, _, classes = mask.shape
        assert classes == len(annotation_classes)

        for c in range(classes):
            contours, hierarchy = cv.findContours(mask[:, :, c], cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

            features.extend(get_features(contours, annotation_classes[c]))

        return FeatureCollection(features)
    else:
        assert len(annotation_classes) == 1

        contours, hierarchy = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
        features = get_features(contours, annotation_classes[0])

        return FeatureCollection(features)

In [ ]:
def get_color():
    r = random.randint(0,255)
    g = random.randint(0,255)
    b = random.randint(0,255)
    return (r,g,b)

def dilate(mask, dilate=False):
    dilated = mask
    nuclei, hierarchy = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    num_dilatations = 0

    while len(nuclei) != 1:
        dilated = cv.dilate(dilated, cv.getStructuringElement(cv.MORPH_ELLIPSE, (10, 10)))
        nuclei, hierarchy = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        num_dilatations += 1
    
    if dilate:
        while num_dilatations:
            dilated = cv2.erode(dilated, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10)))
            num_dilatations -= 1

    return dilated

In [ ]:
def get_annotations(coordinates, gj):
    features = gj['features']

    polygons = dict()
    for a in range(len(coordinates)):
        polygons[a] = []

    for poly in features[1:]:
        akt_poly = copy.deepcopy(poly)
        if len(poly['nucleusGeometry']['coordinates'][0][0]) == 2:
            test_coords_x, test_coords_y = poly['nucleusGeometry']['coordinates'][0][0]
            for idx, (left, bottom, right, top) in enumerate(coordinates):
                if right >= test_coords_x >= left and bottom <= test_coords_y <= top:
                    coordinates_poly = akt_poly['nucleusGeometry']['coordinates']
                    shifted_coords = [(int(coords[0]) - left, int(coords[1]) - bottom) for coords in
                                      coordinates_poly[0]]
                    if shifted_coords[0][0] > 0 and shifted_coords[0][1] > 0:
                        akt_poly['nucleusGeometry']['coordinates'] = [shifted_coords]
                        polygons[idx].append(akt_poly)

    return polygons

In [ ]:
def get_area(contours):
    area = 0
    for contour in contours:
        area += cv.contourArea(contour)
        
    return area

In [ ]:
def ckdnearest(gdA, gdB):
    nA = np.array(list(gdA.geometry.centroid.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.centroid.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=2)

    return dist[:, 1]

In [ ]:
def iterative_dilation(image1, image2, threshold):
    nuclei, hierarchy = cv.findContours(image1, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    area = get_area(nuclei)
    nuclei_count_all = len(nuclei)
    nuclei_count = nuclei_count_all
    nuclei_count_diff = nuclei_count_all
    
    dilated1 = image1
    dilated2 = image2
    while nuclei_count_diff:
        dilated1 = cv.dilate(dilated1, cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5)))
        dilated2 = cv.dilate(dilated2, cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3)))
        
        contours, hierarchy = cv.findContours(dilated1, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        gdf = gpd.GeoDataFrame.from_features(FeatureCollection(get_features(contours, 'other')))
        
        distances = ckdnearest(gdf, gdf)
        if all(distances > threshold):
            break
        
        nuclei_count_diff = nuclei_count - len(contours)
        nuclei_count = len(contours)
    
    return dilated1, dilated2, area, nuclei_count_all

In [ ]:
def get_inflammatory_clustering(image_shape, gj, eps=100, min_samples=20): 
    features = gj['features']
    centroids = list()
    polygons = dict()
    
    index = 0
    for feature in features:
        if feature['properties']['classification']['name'] != 'Region*':
            s = shape(feature['geometry'])
            polygons[index] = s
            centroids.append([s.centroid.x, s.centroid.y])
            index += 1
    
    X = np.array(centroids)
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
    unique = np.unique(db.labels_)
    dilated_img = np.zeros(image_shape).astype(np.uint8)
            
    for unique_idx, unique_value in enumerate(unique[1:]):
        indexes = np.where(db.labels_ == unique_value)[0]
        mask = np.zeros(image_shape, dtype=np.uint8)

        for idx in indexes:
            coors = list(zip(*polygons[idx].exterior.coords.xy))
            pts = [[round(c[0]), round(c[1])] for c in coors]
            cv.fillPoly(mask, [np.array(pts)], 1)

        mask = dilate(mask)
        dilated_img[mask != 0] = 1
        
    return dilated_img

In [ ]:
def get_inflammatory_dilation(image_shape, gj, threshold=0.01, area_threshold=15_000, iteration_threshold=5):
    image_immune = np.zeros(image_shape, dtype=np.uint8)
    image_other = np.zeros(image_shape, dtype=np.uint8)
    
    for feature in gj['features']:
        if feature['properties']['classification']['name'] == 'Immune cells':
            polygon = feature['geometry']
            for coors in polygon['coordinates']:
                pts = [[round(c[0]), round(c[1])] for c in coors]
                cv.fillPoly(image_immune, [np.array(pts)], 1)
        elif feature['properties']['classification']['name'] == 'Other cells' or feature['properties']['classification']['name'] == 'Muscle cells':
            polygon = feature['geometry']
            for coors in polygon['coordinates']:
                pts = [[round(c[0]), round(c[1])] for c in coors]
                cv.fillPoly(image_other, [np.array(pts)], 1)
    
    dilated_immune, dilated_other, area, nuclei_count = iterative_dilation(image_immune, image_other, iteration_threshold)
    mask_immune = np.zeros(image_shape).astype(np.uint8)
    mask_other = np.zeros(image_shape).astype(np.uint8)
    
    return dilated_immune, dilated_other

In [ ]:
def create_mask_from_df(df, size):
    canvas = np.zeros(size).astype(np.uint8)
    for index, row in df.iterrows():
        if row['geometry'].geom_type == 'Polygon':
            polygon = row['geometry']
            coors = list(zip(*polygon.exterior.coords.xy))
            pts = [[round(c[0]), round(c[1])] for c in coors]
            cv.fillPoly(canvas, [np.array(pts)], 1)
    
    return canvas

In [ ]:
def get_final_geojson(df1, df2):
    for index, row in df2.iterrows():
        row_intersections = df1[df1.intersects(row['geometry'].buffer(0))]
        
        if row_intersections is not None and len(list(row_intersections.index)) > 0:
            df1.loc[row_intersections.index, 'geometry'] = df1.loc[row_intersections.index, 'geometry'].union(row['geometry'].buffer(0))
    
    return df1

In [ ]:
def get_mask(img, annotations):
    x, y = int(img.shape[0]), int(img.shape[1])

    mask = np.zeros((x, y), dtype='uint8')
    for feat in annotations:
        classification = feat['properties'].get('classification')
        if classification and classification['name'] == 'Inflammation':
            for coors in feat['geometry']['coordinates']:
                if feat['geometry']['type'] == 'MultiPolygon':
                    coors = coors[0]
                pts = [[round(c[0]), round(c[1])] for c in coors]
                cv.fillPoly(mask, [np.array(pts)], 1)  # fill with ones if cell present

    return mask

In [ ]:
def process_image(image_path, geojson_path, output_path, postprocessing_type, algo_type, iteration_threshold, area_threshold):
    #javabridge.start_vm(class_path=bf.JARS)
    set_logs()

    image_reader = bf.formatreader.make_image_reader_class()()
    gj = get_geojson_from_file(geojson_path)
    
    file_name = image_path.replace('\\', '/').split('/')[-1].split('.')[0]
    
    if gj is None:
        print('Error')
        return None
    
    features_all, geojson_cell = get_fragment_coords(gj)

    image_reader.allowOpenToCheckType(True)
    image_reader.setId(image_path)
    image_reader.setSeries(0)
    wrapper = bf.formatreader.ImageReader(path=image_path, perform_init=False)
    wrapper.rdr = image_reader

    sizeX = wrapper.rdr.getSizeX()
    sizeY = wrapper.rdr.getSizeY()
    
    if algo_type == 'dbscan':
        mask = get_inflammatory_clustering((sizeY, sizeX), gj)
        geojson_file = create_geojson(mask, ['inflammations'])
        
        with open(f'{output_path}/{file_name}.geojson', 'w') as f:
            dump(geojson_file, f)
        
    elif algo_type == 'dilatation':
        mask1, mask2 = get_inflammatory_dilation((sizeY, sizeX), gj, iteration_threshold=iteration_threshold, area_threshold=area_threshold)
        geojson_file1 = create_geojson(mask1, ['inflammations'])
        geojson_file2 = create_geojson(mask2, ['inflammations'])
        
        df1 = gpd.GeoDataFrame.from_features(geojson_file1)
        df2 = gpd.GeoDataFrame.from_features(geojson_file2)
        if not df1.empty:
            df = get_final_geojson(df1, df2)
            df = df[df.area > area_threshold]
        
            mask = create_mask_from_df(df, (sizeY, sizeX))
        else:
            mask = mask1
        
        with open(f'{output_path}/{file_name}-{iteration_threshold}-{area_threshold}.geojson', 'w') as f:
            dump(create_geojson(mask, ['inflammations']), f)
        
        o_mask = geojson.load(open('D:/Master Thesis/Data/21.06. 2022 annotations/1225_21_HE.vsi - 20x_annotations.geojson'))
        image = cv.imread(f"D:/Master Thesis/ANN Imunne cells/1225_21_HE.png")
        original_mask = get_mask(image, o_mask['features'])
        IoU, dice = evaluate(original_mask, mask)
        print(f'IoU: {IoU}, DICE: {dice}')
    
    print(f'{file_name} done!')
        
    #javabridge.kill_vm()
    

In [ ]:
javabridge.start_vm(class_path=bf.JARS)
for th in [25, 35, 45, 50, 60, 65, 75, 80, 100]:
    for area_th in [15_000, 20_000, 25_000, 30_000, 35_000, 40_000]:
        print(f'Area: {area_th}, Distance: {th}')
        process_image(
            image_path='D:/Master Thesis/Data/EMB-IKEM-2022-03-09/1225_21_HE.vsi',
            geojson_path='D:/Master Thesis/Data/EMB-IKEM-2022-03-09/QuPath project EMB - anotations/annotations/1225_21_HE.vsi - 20x.geojson',
            output_path='D:/test',
            postprocessing_type='all',
            algo_type='dilatation',
            iteration_threshold=th,
            area_threshold=area_th
        )
javabridge.kill_vm()